# Basic Usage

In [3]:
from pydantic import BaseModel, ValidationError

class User(BaseModel):
    id: int 
    name: str
    email: str

# Model validation
try:
    user = User(id=1, name='John Doe', email='john.doe@example.com')
    print(user)
except ValidationError as e:
    print(e.json())

id=1 name='John Doe' email='john.doe@example.com'


## Nested Models

In [7]:
from pydantic import EmailStr

class Address(BaseModel):
    street: str
    city: str
    zip_code: str
    
class UserWithAddress(BaseModel):
    id: int
    name: str
    email: EmailStr
    address: Address
    
# Create an instance with a nested model
user_with_address = UserWithAddress(
    id=1,
    name='John Doe',
    email='john.doe@example.com',
    address={'street': '123 Main St', 'city': 'Anytown', 'zip_code': '12345'}
)
print(user_with_address)
    

id=1 name='John Doe' email='john.doe@example.com' address=Address(street='123 Main St', city='Anytown', zip_code='12345')


## Default Values and Optional Fields

In [8]:
from typing import Optional

class UserWithOptional(BaseModel):
    id: int 
    name: str
    email: Optional[EmailStr] = None # Default values is None
    
user = UserWithOptional(id=1, name='John Doe')
print(user)

id=1 name='John Doe' email=None


## Setting Management

In [14]:
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    app_name: str
    admin_email: EmailStr
    
    class Config:
        env_file = '.env' # load environment variables from a .env file
    
settings = Settings(app_name='Jarvis', admin_email='admin@example.com')
print(setting.app_name)

ValidationError: 3 validation errors for Settings
auth_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='asbhajKBBSda', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
my_api_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='DHbsdajhbw1234', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
env2
  Extra inputs are not permitted [type=extra_forbidden, input_value='https://envurl.com', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden

### Serialization and Deserialization

In [16]:
user_dict = user.model_dump()
print(user_dict)

user_json = user.model_dump_json()
print(user_json)

{'id': 1, 'name': 'John Doe', 'email': None}
{"id":1,"name":"John Doe","email":null}


### Custom Validator

In [25]:
from pydantic import field_validator

class User(BaseModel):
    id: int
    name: str
    email: EmailStr
    
    @field_validator('email')
    def email_must_contain_at(cls, v):
        if '@' not in v:
            raise ValueError('Email must contain @')
        return v
try:
    user = User(id=1, name='John Doe', email='johndoeexample.com')
except ValidationError as e:
    print(e)

1 validation error for User
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='johndoeexample.com', input_type=str]


### Field Customization

In [38]:
from pydantic import BaseModel, Field

class User(BaseModel):
    id: int 
    name: str = Field(..., description='The name of the user', max_length=50)
    email: str = Field(..., description='The email of the user', patern=r'^[\w\.-]+@[\w\.-]+\.[a-zA-Z]{2,}$')
    # email: EmailStr = Field(..., description='The email of the user')
    
user = User(id=1, name='John Doe', email='johndoe@example.com')
print(user)

id=1 name='John Doe' email='johndoe@example.com'


### Model Inheritance


In [39]:
class BaseUser(BaseModel):
    id: int
    name: str

class AdminUser(BaseUser):
    admin_level: int
    
admin = AdminUser(id=1, name='Admin', admin_level=5)
print(admin)

id=1 name='Admin' admin_level=5


### Config Class

In [44]:
from pydantic import BaseModel
from enum import Enum

class User(BaseModel):
    id: int
    name: str
    
    class Config:
        populate_by_name = True
        use_enum_values = True # Use enum values instead of enum instances
    
class UserRole(str, Enum):
    admin = 'admin'
    user = 'user'

class UserWithRole(BaseModel):
    id: int
    name: str
    role: UserRole

user = UserWithRole(id=1, name='John Doe', role='admin')
print(user)

id=1 name='John Doe' role=<UserRole.admin: 'admin'>


### Serialization Options

In [47]:
class User(BaseModel):
    id: int
    name: str
    email: str
    
user = User(id=1, name='John Doe', email='johndoe@example.com')
user_json = user.model_dump_json(exclude={'email'})
print(user_json)

{"id":1,"name":"John Doe"}


# Common Use Cases

1. API Data Validation: Pydantic is widely used in web frameworks like FastAPI for vlidating request and response data

In [49]:
!pip install fastapi

In [51]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    price: float
    is_offer: bool = Field(default=None)
    
@app.post('/items/')
async def create_item(item: Item):
    return item

2. Configuration Management: Pydantic can be used to manage application settings, loading then from environment variables or configuration files.

In [54]:
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    database_url: str
    secret_key: str
    
    class Config:
        env_file = '.env'
    
settings = Settings()
print(settings.database_url)

ValidationError: 5 validation errors for Settings
database_url
  Field required [type=missing, input_value={'auth_key': 'asbhajKBBSd...': 'https://envurl.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
secret_key
  Field required [type=missing, input_value={'auth_key': 'asbhajKBBSd...': 'https://envurl.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
auth_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='asbhajKBBSda', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
my_api_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='DHbsdajhbw1234', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
env2
  Extra inputs are not permitted [type=extra_forbidden, input_value='https://envurl.com', input_type=str]
    For further information visit https

3. Data Transformation: Pydantic can be used to transform and validate data coming from various sources, such as database or external APIs.

In [55]:
from typing import List

class Product(BaseModel):
    id: int
    name: str
    price: float

class Order(BaseModel):
    products: List[Product]

order_data = {
    'products': [
        {'id': 1, 'name': 'Product 1', 'price': 10.0},
        {'id': 2, 'name': 'Product 2', 'price': 20.0},
    ]
}

order = Order(**order_data)
print(order)

products=[Product(id=1, name='Product 1', price=10.0), Product(id=2, name='Product 2', price=20.0)]


# Advanced Concepts

1. Model Composition: Pydantic allows you to compose models using other models as fields. This is useful for creating complex data structures while maintaining clarity.

In [57]:
from typing import List

class Address(BaseModel):
    street: str
    city: str
    zip_code: str
    
class User(BaseModel):
    id: int
    name: str
    addresses: List[Address]
    
user = User(
    id=1,
    name='John Doe',
    addresses=[
        Address(street='123 Main St', city='Anytown', zip_code='12345'),
        Address(street='456 Elm St', city='Othertown', zip_code='67890'),
    ]
)
print(user.model_dump())

{'id': 1, 'name': 'John Doe', 'addresses': [{'street': '123 Main St', 'city': 'Anytown', 'zip_code': '12345'}, {'street': '456 Elm St', 'city': 'Othertown', 'zip_code': '67890'}]}


2. Custom Data Types: You can create custom data types by subclassing `pydantic.BaseModel` or using `pydantic.constr`, `pydantic.conint`, etc., to enfore constraints on built-in types.

In [63]:
from pydantic import BaseModel, conint, constr

class User(BaseModel):
    id: conint(gt=0) # id must be greater than 0
    name: constr(min_length=1) # name must be at least 1 character long
    email: constr(pattern=r'^[\w\.-]+@[\w\.-]+\.[a-zA-Z]{2,}$') # email must match the regex pattern

user = User(id=1, name='John Doe', email='john.doe@example.com')
print(user.model_dump())

{'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'}


In [67]:
from pydantic import BaseModel, Field, EmailStr

class User(BaseModel):
    id: int = Field(..., ge=1) # id must be greater than or equal to 1
    name: str = Field(..., min_length=1) # name must be at least 1 character long
    email: EmailStr
    
user = User(id=1, name='John Doe', email='john.doe@example.com')
print(user.model_dump())

{'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'}


3. Field Aliases: You can use field aliases to map model attributes to different names, which is useful when working with external APIs or databases.

In [73]:
class User(BaseModel):
    user_id: int = Field(..., alias='id')
    name: str
    email: EmailStr

user_data = {'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'}
user = User(**user_data)
print(user.model_dump(by_alias=True))

{'id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'}


4. JSON Encoding and Decoding: Pydantic provides built-in supports for JSON encoding and decoding, allowing you to easily convert models to and from JSON.

In [79]:
import json

user = User(id=1, name='John Doe', email='john.doe@example.com')
json_data = user.model_dump_json(by_alias=True)
print(json_data)

user_from_json = User.model_validate_json(json_data)
print(user_from_json.model_dump())

{"id":1,"name":"John Doe","email":"john.doe@example.com"}
{'user_id': 1, 'name': 'John Doe', 'email': 'john.doe@example.com'}


5. Environment Variable Loading: Pydantic can load settings from environment variables, making it easy to manage configuration in different environments (development, testing, production).

In [82]:
from pydantic_settings import BaseSettings
from pydantic import HttpUrl

class Settings(BaseSettings):
    database_url: HttpUrl
    debug: bool = False

    class Config:
        env_file = '.env' # load from .env file

settings = Settings()
print(settings.database_url)

ValidationError: 4 validation errors for Settings
database_url
  Field required [type=missing, input_value={'auth_key': 'asbhajKBBSd...': 'https://envurl.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
auth_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='asbhajKBBSda', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
my_api_key
  Extra inputs are not permitted [type=extra_forbidden, input_value='DHbsdajhbw1234', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
env2
  Extra inputs are not permitted [type=extra_forbidden, input_value='https://envurl.com', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden

### Performance Considerations

* Validation Overhead: While Pydantic provides powerful validation features, it can introduce overhead, especially with large models or complex nested structures. Always profile your application if performance is a concern.
* User `@root_validator` for Cross-Field Validation: If you need to validate multiple fields together, use the `@root_validator` decorator. This can help reduce the number of validation passes.

In [94]:
from pydantic import model_validator

class User(BaseModel):
    password: str
    confirm_password: str
    
    @model_validator(mode='after')
    @classmethod
    def check_passwords_match(cls, values):
        values = values.model_dump()
        if values.get('password') != values.get('confirm_password'):
            raise ValueError('Passwords do not match')
        return values
    
# Example Usage

user = User(password='123', confirm_password='123')
print(user)

password='123' confirm_password='123'


C:\Users\aldir\AppData\Local\Temp\ipykernel_11696\1820515365.py:17: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  user = User(password='123', confirm_password='123')


# Integration with Other Libraries

1. FastAPI: Pydantic is heavily integrated with FastAPI, a modern web framework for building APIs with Python. FastAPI uses Pydantic models for request validation and response serialization. 

In [95]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    price: float
    
@app.post('/items/')
async def create_item(item: Item):
    return item

2. SQLAlchemy: You can use Pydantic with SQLAlchemy to validate data before inserting it into the database. This can help ensure that only valid data is stored.

In [101]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker, declarative_base

Base = declarative_base()

class UserModel(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    email = Column(String)
  
# Create an engine that stores data in the local directory
# sqlalchemy_e  
# sqlalchemy_engine = create_engine('sqlite:///database.db'

engine = create_engine('sqlite:///database.db')

# Create all tables in the engine. This is equivalent to "Create Table"
# statements in raw SQL.
Base.metadata.create_all(bind=engine)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

class User(BaseModel):
    name: str
    email: EmailStr
    
# Example of inserting a user
def create_user(user: User):
    db = SessionLocal()
    db_user = UserModel(name=user.name, email=user.email)
    db.add(db_user)
    db.commit()
    db.refresh(db_user)
    return db_user

create_user(User(name='John Doe', email='john@example.com'))

In [108]:
# Assuming you have already created the engine and the table

# Create a new session
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

db = SessionLocal()

# Retrieve all users
users = db.query(UserModel).all()

# Print the users
for user in users:
    print(f"ID: {user.id}, Name: {user.name}, Email: {user.email}")

# Close the session
db.close()


ID: 1, Name: John Doe, Email: john@example.com


3. Data Science and Machine Learning: Pydantic can be used to validate and manage configurations for machine learning models, ensuring that the parameters passed to models are valid.

In [111]:
class ModelConfig(BaseModel):
    learning_rate: float
    batch_size: int
    num_epochs: int
    
config = ModelConfig(learning_rate=0.001, batch_size=32, num_epochs=10)
print(config)

learning_rate=0.001 batch_size=32 num_epochs=10


# Real-World Example

Here's a more comprehensive example that combines several features of Pydantic, including nested models, custom validators, and environment variable loading:

In [115]:
from pydantic import BaseModel, EmailStr, field_validator
from pydantic_settings import BaseSettings
from typing import List

class Address(BaseModel):
    street: str
    city: str
    zip_code: str

class User(BaseModel):
    id: int
    name: str
    email: EmailStr
    addresses: List[Address]
    
    @field_validator('name')
    @classmethod
    def name_must_not_be_empty(cls, v):
        if not v.strip():
            raise ValueError('Name must not be empty')
        return v

class Settings(BaseSettings):
    app_name: str
    admin_email: EmailStr
    
    class Config:
        env_file = '.env'

# Example usage
# settings = Settings()
# print(settings.app_name)

user = User(
    id=1,
    name='John Doe',
    email='john.doe@example.com',
    addresses=[
        Address(street='123 Main St', city='Anytown', zip_code='12345'),
    ]
)
print(user)

id=1 name='John Doe' email='john.doe@example.com' addresses=[Address(street='123 Main St', city='Anytown', zip_code='12345')]


# Conclusion

Pydantic is a versatile and powerful library that enhances data validation and settings management in Python applications. Its integration with type annotations, support for complex data structures, and ability to work seamlessly with frameworks like FastAPI and libraries like SQLAlchemy make it an invaluable tool for developers. By leveraging Pydantic's features, you can ensure data integrity, improve code readability, and simplify configuration management in your applications. Whether you're building APIs, managing application settings, or validating complex data structures, Pydantic provides the tools you need to do so effectively.